In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats

import matplotlib.pyplot
import pandas as pd
import time

In [6]:
nba_teams = teams.get_teams() 

def get_team_stats_by_year(team_id):
    team_stats = teamyearbyyearstats.TeamYearByYearStats(team_id=team_id)
    return team_stats.get_data_frames()[0]
data = pd.DataFrame()
for team in nba_teams:
    team_data = get_team_stats_by_year(team['id'])

    team_data = team_data[team_data['YEAR'] >= '2000']
    data = pd.concat([data, team_data])

In [7]:
data

,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,PO_WINS,PO_LOSSES,CONF_COUNT,DIV_COUNT,NBA_FINALS_APPEARANCE,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
51,1610612737,Atlanta,Hawks,2000-01,82,25,57,0.305,13,7,0,0,15.0,8,N/A,2876,6668,0.431,333,933,0.357,1374,1811,0.759,1028,2490,3518,1559,1863,634,1368,387,7459,26
52,1610612737,Atlanta,Hawks,2001-02,82,33,49,0.402,12,6,0,0,15.0,8,N/A,2901,6610,0.439,423,1194,0.354,1486,1942,0.765,955,2445,3400,1656,1702,667,1275,350,7711,19
53,1610612737,Atlanta,Hawks,2002-03,82,35,47,0.427,11,5,0,0,15.0,8,N/A,2859,6434,0.444,402,1141,0.352,1594,2011,0.793,937,2558,3495,1679,1783,611,1367,473,7714,18
54,1610612737,Atlanta,Hawks,2003-04,82,28,54,0.341,12,7,0,0,15.0,8,N/A,2829,6529,0.433,419,1249,0.335,1534,1976,0.776,996,2507,3503,1648,1826,627,1350,408,7611,15
55,1610612737,Atlanta,Hawks,2004-05,82,13,69,0.159,15,5,0,0,15.0,5,N/A,2942,6672,0.441,304,973,0.312,1417,1994,0.711,1100,2335,3435,1614,2009,629,1319,344,7605,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,1610612766,Charlotte,Hornets,2019-20,65,23,42,0.354,10,4,0,0,15.0,5,N/A,2425,5586,0.434,785,2231,0.352,1052,1406,0.748,715,2066,2781,1549,1223,428,949,268,6687,30
30,1610612766,Charlotte,Hornets,2020-21,72,33,39,0.458,0,0,0,0,15.0,5,N/A,2875,6324,0.455,985,2666,0.369,1146,1505,0.761,762,2389,3151,1933,1298,565,1069,344,7881,23
31,1610612766,Charlotte,Hornets,2021-22,82,43,39,0.524,10,3,0,0,15.0,5,N/A,3508,7497,0.468,1143,3130,0.365,1298,1753,0.740,888,2767,3655,2302,1629,707,1087,402,9457,4
32,1610612766,Charlotte,Hornets,2022-23,82,27,55,0.329,14,5,0,0,15.0,5,N/A,3385,7413,0.457,881,2669,0.330,1447,1933,0.749,901,2751,3652,2062,1661,634,1164,425,9098,27


In [9]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from datetime import datetime

# Set the season type to 'Playoffs'
season_type = 'Playoffs'

# Get the game logs for the playoffs
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable=season_type)
games = gamefinder.get_data_frames()[0]

# Filter the game logs to only include games from 2020 onwards
games['GAME_DATE'] = games['GAME_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
games_since_2000 = games[(games['GAME_DATE'].dt.year >= 2000)] #& (games['GAME_DATE'].dt.year <= 2020)]

In [10]:
games_since_2000

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42023,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,L,240,88,...,0.538,7,28,35,18,4,4,13,20,-18.0
1,42023,1610612738,BOS,Boston Celtics,0042300405,2024-06-17,BOS vs. DAL,W,240,106,...,0.850,15,36,51,25,9,2,7,15,18.0
2,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,239,84,...,0.923,4,27,31,18,2,5,13,19,-38.0
3,42023,1610612742,DAL,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,W,240,122,...,0.682,13,39,52,21,7,2,8,17,38.0
4,42023,1610612742,DAL,Dallas Mavericks,0042300403,2024-06-12,DAL vs. BOS,L,240,99,...,0.875,7,36,43,15,5,1,8,17,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5547,41999,1610612755,PHI,Philadelphia 76ers,0049900004,2000-04-22,PHI @ CHH,W,234,92,...,0.710,14,22,36,14,8,6,10,27,10.0
5548,41999,1610612759,SAS,San Antonio Spurs,0049900002,2000-04-22,SAS vs. PHX,L,236,70,...,0.571,9,41,50,15,7,9,18,19,-2.0
5549,41999,1610612762,UTA,Utah Jazz,0049900003,2000-04-22,UTA vs. SEA,W,234,104,...,0.885,15,35,50,26,9,6,14,24,11.0
5550,41999,1610612756,PHX,Phoenix Suns,0049900002,2000-04-22,PHX @ SAS,W,235,72,...,0.900,10,37,47,20,10,10,14,20,2.0
